In [1]:
%load_ext autoreload
%autoreload 2

%connect_info

{
  "shell_port": 33943,
  "iopub_port": 59575,
  "stdin_port": 34245,
  "control_port": 49695,
  "hb_port": 34185,
  "ip": "127.0.0.1",
  "key": "729750d7-44ad008a02b71d62c62fed5f",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-293b9907-f497-48c9-9c65-7ba91bb8319f.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
import django

In [3]:
django.setup()

No module named 's3_uploader.conf'


In [4]:
"""
Project Configuration Sheets
"""
FIN_MAN = "FIN_MAN"
FIN_MAN_PROJECT_SHEET_NAME = "Finman Project Configuration"
FIN_MAN_CONFIGURATION_SHEET_NAME="FinMan Configuration_v4 (Payment Request Complete Config)"
VENDOR_CONFIGURATION_SHEET_NAME = "Vendor Configuration_v1 - Test"

In [5]:
"""
Populate project and project roles
"""

from django.db import transaction

@transaction.atomic()
def populate_project_and_roles():
    from ib_iam.populate.populate_utils import populate_projects
    populate_projects(spread_sheet_name=FIN_MAN_PROJECT_SHEET_NAME)

    from ib_tasks.populate.population_utils import populate_project_roles
    populate_project_roles(FIN_MAN_CONFIGURATION_SHEET_NAME)

In [6]:
"""
Populate tasks related data
"""

from django.db import transaction


@transaction.atomic()
def populate_task_and_boards_related_data():
    
    project_configuration_sheets = [
        (FIN_MAN_CONFIGURATION_SHEET_NAME, FIN_MAN)
    ]

    for project_configuration_sheet in project_configuration_sheets:
        configuration_sheet, project_id = project_configuration_sheet
        from ib_tasks.populate.population_utils import populate_data
        populate_data(project_id, configuration_sheet)
        
        from ib_boards.populate.get_data_from_sheet_for_boards_and_columns import GetBoardsAndColumnsDataFromSheet
        loader = GetBoardsAndColumnsDataFromSheet()
        loader.get_data_from_sheet(configuration_sheet)
        loader.create_boards_and_columns(configuration_sheet)

In [7]:
"""
Populate project related data
"""
from django.db import transaction

@transaction.atomic()
def populate_project_related_data():

    from ib_tasks.populate.population_utils import populate_projects_for_task_templates
    populate_projects_for_task_templates(FIN_MAN_PROJECT_SHEET_NAME)
    
    from ib_boards.populate.get_data_from_sheet_for_project_boards import GetSheetDataForProjectBoards
    loader = GetSheetDataForProjectBoards()
    loader.get_data_from_project_boards_sub_sheet(FIN_MAN_PROJECT_SHEET_NAME)

In [8]:
populate_project_and_roles()
populate_task_and_boards_related_data()
populate_project_related_data()

SpreadsheetNotFound: 

In [10]:
from ib_tasks.populate.write_stage_actions import writing_data_to_stage_actions_logic
writing_data_to_stage_actions_logic()

[none] [ib_workflows_backend - beta] INFO     [12:07:54]  [/usr/lib/python3.8/lib2to3/pgen2/driver.py] [driver.py][load_grammar] [120]: Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
[none] [ib_workflows_backend - beta] INFO     [12:07:54]  [/usr/lib/python3.8/lib2to3/pgen2/driver.py] [driver.py][load_grammar] [120]: Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt


LE: Starting Logentries Asynchronous Socket Appender


In [11]:
"""
Populate Ib IAM Related Data For FinMan Project Configuration

Note:- While Running the this below script need to Run the ELASTIC SEARCH SERVER
"""
from ib_iam.populate.populate_utils import populate
populate(FIN_MAN_CONFIGURATION_SHEET_NAME)

from ib_users.models import *
UserAccount.objects.create_superuser(username='admin', password='admin@123', email='admin@example.com')
UserAccount.objects.create_superuser(username='fduser', password='fduser@123', email='fduser@example.com')

[none] [ib_workflows_backend - beta] INFO     [12:07:58]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.038s]
[none] [ib_workflows_backend - beta] INFO     [12:07:58]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.027s]
[none] [ib_workflows_backend - beta] INFO     [12:07:59]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 reque


Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=xL91DbPg9YdVtnjgsDhT2v4Xxzylv4">Verify Your Email</a><br>
<br>
Thank you
 asd
template_:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=xL91DbPg9YdVtnjgsDhT2v4Xxzylv4">Verify Your Email</a><br>
<br>
Thank you

serializer_template:  <django.template.base.Template object at 0x7f408996d3a0>
data:  {}
html_content:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=xL91DbPg9YdVtnjgsDhT2v4Xxzylv4">Verify Your Email</a><br>
<br>
Thank you



[none] [ib_workflows_backend - beta] INFO     [12:08:01]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.021s]



Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=FxJg4oAxRAZYhJEgTCwWYCVxFvvlOw">Verify Your Email</a><br>
<br>
Thank you
 asd
template_:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=FxJg4oAxRAZYhJEgTCwWYCVxFvvlOw">Verify Your Email</a><br>
<br>
Thank you

serializer_template:  <django.template.base.Template object at 0x7f40897a0790>
data:  {}
html_content:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=FxJg4oAxRAZYhJEgTCwWYCVxFvvlOw">Verify Your Email</a><br>
<br>
Thank you



[none] [ib_workflows_backend - beta] INFO     [12:08:03]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.022s]


...

[none] [ib_workflows_backend - beta] INFO     [12:08:23]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.020s]



Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=ek3s4QzX2yXeyEDRsgC8jd1isY4uTt">Verify Your Email</a><br>
<br>
Thank you
 asd
template_:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=ek3s4QzX2yXeyEDRsgC8jd1isY4uTt">Verify Your Email</a><br>
<br>
Thank you

serializer_template:  <django.template.base.Template object at 0x7f4089747610>
data:  {}
html_content:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=ek3s4QzX2yXeyEDRsgC8jd1isY4uTt">Verify Your Email</a><br>
<br>
Thank you



[none] [ib_workflows_backend - beta] INFO     [12:08:25]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/user-beta/_doc [status:201 request:0.019s]



Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=slcMA2oxUDuJ5BkSGxCIk6MdO3cm4e">Verify Your Email</a><br>
<br>
Thank you
 asd
template_:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=slcMA2oxUDuJ5BkSGxCIk6MdO3cm4e">Verify Your Email</a><br>
<br>
Thank you

serializer_template:  <django.template.base.Template object at 0x7f4089756af0>
data:  {}
html_content:  
Hi 
Verify your email by using below link,
<br>
<a href="https://ib-workflows-web-alpha.apigateway.in/verify-email?token=slcMA2oxUDuJ5BkSGxCIk6MdO3cm4e">Verify Your Email</a><br>
<br>
Thank you



<UserAccount: e2897075-efb4-4d94-8bdd-1d646798a852>

In [12]:
"""
Populate Elastic Search Data For Cities

"""
from ib_iam.populate.populate_elastic_search_data import populate_data
populate_data()

[none] [ib_workflows_backend - beta] INFO     [12:08:28]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/country-beta/_doc [status:201 request:0.022s]
[none] [ib_workflows_backend - beta] INFO     [12:08:28]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/country-beta/_doc [status:201 request:0.024s]
[none] [ib_workflows_backend - beta] INFO     [12:08:28]  [/home/ubuntu/ib-workflows-backend/venv/lib/python3.8/site-packages/elasticsearch/connection/base.py] [base.py][log_request_success] [227]: POST https://search-ib-workflows-dev-nck6c5mhnwbsg2ajsnmyobophm.ap-south-1.es.amazonaws.com:443/country-beta/_doc [status:

In [3]:
from elasticsearch_dsl import connections
from django.conf import settings
es = connections.create_connection(
hosts=[settings.ELASTICSEARCH_ENDPOINT], timeout=20)

In [4]:
from ib_tasks.documents.elastic_task import TASK_INDEX_NAME
es.indices.create(index = TASK_INDEX_NAME)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'task-beta'}